# ControlFlow Customer Call Routing Example

This example was copied and modified from https://controlflow.ai/examples/call-routing.

In [ ]:
from enum import Enum
import random

import controlflow as cf

In [ ]:
import os
from langchain_openai import ChatOpenAI

# set the default model
cf.default_model = ChatOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
    temperature=0.0,
)

assert cf.Agent('Marvin').get_model().openai_api_base == os.environ.get('OPENAI_BASE_URL'), f"{cf.Agent('Marvin').get_model().openai_api_base} != {os.environ.get('OPENAI_BASE_URL')}"

In [ ]:
class Department(Enum):
    BILLING = "billing"
    RETURNS = "returns"
    SALES = "sales"
    SUPPORT = "support"
    TECHNICAL = "technical"

In [ ]:
@cf.flow
def routing_flow():
    department = random.choice(list(Department))

    # create an agent to be our "customer"
    customer = cf.Agent(
        "Customer",
        instructions=f"""
            You are training customer reps by pretending to be a customer
            calling into a call center. You need to be routed to the
            {department} department. Come up with a good backstory.""",
    )

    # assert customer.get_model().openai_api_base == os.environ.get('OPENAI_BASE_URL'), f"{customer.get_model().openai_api_base} != {os.environ.get('OPENAI_BASE_URL')}"

    # create an agent to be our "trainee"
    trainee = cf.Agent(
        "Trainee",
        instructions="""
            You are a trainee customer service representative. You need to
            listen to the customer's story and route them to the correct
            department. Note that the customer is another agent training you.""",
    )

    # assert trainee.get_model().openai_api_base == os.environ.get('OPENAI_BASE_URL'), f"{trainee.get_model().openai_api_base} != {os.environ.get('OPENAI_BASE_URL')}"

    # create a task for the agents to perform
    task = cf.Task(
        """
        In this task, the customer agent and the trainee agent will speak to
        each other. The customer is trying to be routed to the correct
        department. The trainee will listen and ask questions to determine the
        correct department. 
        """,
        instructions="""
        Only the trainee can mark the task successful by routing the customer to
        the correct department. The trainee must begin the conversation by
        greeting the customer. Agents speak to each other by posting messages
        directly to the thread. Do not use the `end_turn` tool or try to talk 
        to a user.
        """,
        agents=[trainee, customer],
        result_type=Department,
    )

    routed_department = task.run()

    if routed_department == department:
        print("Success! The customer was routed to the correct department.")

    else:
        print(
            "Failed. The customer was routed to the wrong department. "
            f"The correct department was {department}."
        )

In [ ]:
if __name__ == "__main__":
    routing_flow()